In [1]:
import copy
import sys
import numpy as np
import open3d as o3d
sys.path.append(".")
import pyransac3d as pyrsc
import flattten_pcd as fp
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [22]:
center = [[]]
center[0] = [1,2,3]
center[1] = [2,3,4]
print(center)

IndexError: list assignment index out of range

In [6]:
load_pcd = o3d.io.read_point_cloud("point_pipe.pcd")
print("length pcd ", len(np.array(load_pcd.points)))
o3d.visualization.draw_geometries([load_pcd])

length pcd  15015


In [7]:
print("Downsample the point cloud with a voxel of 0.02")
pcd2 = load_pcd.voxel_down_sample(voxel_size=0.02)
print("length downpcd ", len(np.array(pcd2.points)))
o3d.visualization.draw_geometries([pcd2])

Downsample the point cloud with a voxel of 0.02
length downpcd  3635


In [8]:
plane_model, inliers = pcd2.segment_plane(distance_threshold=0.05,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd2.select_by_index(inliers)
print(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd2.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([0, 1.0, 0])
print(len(inliers), len(np.array(pcd2.points)))

Plane equation: 0.01x + 0.00y + 1.00z + 0.00 = 0
[0, 3, 4, 5, 6, 7, 8, 9, 10, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 87, 88, 89, 90, 91, 92, 101, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 119, 120, 121, 128, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 265, 266, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301

In [9]:
print("inlider")
o3d.visualization.draw_geometries([inlier_cloud])
print("outlier_cloud")
o3d.visualization.draw_geometries([outlier_cloud])

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(outlier_cloud.cluster_dbscan(eps=0.1, min_points=100, print_progress=True))

inlider
outlier_cloud
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 1
Precompute neighbors.[========================================] 100%============================>    ] 87%


In [10]:
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
print("length label:  ",len(labels))
print("length outlier_cloud:  ",len(np.array(outlier_cloud.points)))
o3d.visualization.draw_geometries([outlier_cloud])

point cloud has 1 clusters
length label:   1190
length outlier_cloud:   1190


In [11]:
cluster = o3d.geometry.PointCloud()
for c in range(max_label+1):
    cluster = [i for i, n in enumerate(labels.tolist()) if n == c]
    print(cluster) 
    cluster = outlier_cloud.select_by_index(cluster)
    print("length cluster:  ",len(np.array(cluster.points)))
    o3d.visualization.draw_geometries([cluster])

[1, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 42, 43, 44, 51, 53, 54, 55, 56, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 124, 125, 126, 127, 128, 129, 130, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 150, 156, 157, 164, 165, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 192, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 204, 206, 207, 208, 209, 213, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 256, 257, 258, 259, 260, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 279, 280, 281, 282, 283, 284, 295, 311, 315, 316, 317, 318, 319, 320, 322, 323, 324, 325, 327, 328, 329, 330, 332, 333, 345, 346, 352, 358, 360, 361, 362, 363, 364, 365, 371, 372, 375, 376, 378, 379, 380, 382, 383, 390, 393, 400, 403, 410, 413, 416, 419, 427, 428, 429, 430, 431, 432, 433, 434, 435, 437, 439, 441, 442, 443, 444, 445

In [12]:
cluster = [i for i, n in enumerate(labels.tolist()) if n == 1]
print(cluster) 
cluster = outlier_cloud.select_by_index(cluster)
print("length cluster:  ",len(np.array(cluster.points)))
o3d.visualization.draw_geometries([cluster])

[]
length cluster:   0


In [13]:
import cylinder2 as cy2

In [15]:
cil = cy2.Cylinder()
points = np.asarray(cluster.points)
center, normal, radius, inliers = cil.fit(points, thresh=0.01, maxIteration=10000)

#center, normal, radius, inliers = fp.get_cylinder(cluster, thresh=0.01, maxIteration=10000)
mesh_cylinder = fp.get_clylinder_mesh(center, normal, radius)
print("center: " + str(center))
print("radius: " + str(radius))
print("vecC: " + str(normal))
#print("inliers: ", inliers)

inlier_cloud = cluster.select_by_index(inliers)

ValueError: Sample larger than population or is negative

In [ ]:
o3d.visualization.draw_geometries([mesh_cylinder,inlier_cloud])

In [158]:
import numpy as np

a = np.arange(5, 15) # array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

b=np.where(a > 10)[0] # 6 ~ 9번 위치가 만족 -> (array([6, 7, 8, 9]),)

print(a)
print(b)
a = np.array([[15, 8, 12], [11, 7, 3]])
c=np.where(a > 10) # (array([0, 0, 1]), array([0, 2, 0]))
print (a)
print(c)

[ 5  6  7  8  9 10 11 12 13 14]
[6 7 8 9]
[[15  8 12]
 [11  7  3]]
(array([0, 0, 1]), array([0, 2, 0]))
